In [ ]:
Updated code which takes screenshot at proper interval and saved to cloud.

In [ ]:
import logging
import time
import threading
import os
import pyautogui
import boto3
from datetime import datetime
from pynput import mouse, keyboard
from pystray import Icon, MenuItem, Menu
from PIL import Image, ImageDraw
from botocore.exceptions import NoCredentialsError, PartialCredentialsError

# Configure logging for activity tracking
logging.basicConfig(filename="activity_log.txt", level=logging.INFO)

# Global variable to track activity status
is_active = False

# Function to create an icon for the system tray
def create_image():
    # Create an image with PIL for the tray icon
    width = 64
    height = 64
    image = Image.new("RGB", (width, height), (255, 255, 255))
    draw = ImageDraw.Draw(image)
    draw.rectangle([0, 0, width, height], fill=(0, 128, 255))
    return image

# Function to log user activity
def log_activity(action_type, action_detail):
    timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
    logging.info(f"{timestamp} - {action_type}: {action_detail}")

# Mouse listener functions
def on_move(x, y):
    global is_active
    if not is_active:
        log_activity("Mouse Move", f"Mouse moved to {x}, {y}")
        is_active = True

def on_click(x, y, button, pressed):
    if pressed:
        log_activity("Mouse Click", f"Mouse clicked at {x}, {y}")

def on_scroll(x, y, dx, dy):
    log_activity("Mouse Scroll", f"Mouse scrolled at {x}, {y}, delta: ({dx}, {dy})")

# Keyboard listener functions
def on_press(key):
    log_activity("Key Pressed", f"Key {key} pressed")

# Function to stop the listener when closing the tray icon
def on_quit(icon, item):
    icon.stop()

# Function to start the background listener (mouse and keyboard)
def start_listeners():
    # Start mouse listener
    mouse_listener = mouse.Listener(on_move=on_move, on_click=on_click, on_scroll=on_scroll)
    mouse_listener.start()

    # Start keyboard listener
    keyboard_listener = keyboard.Listener(on_press=on_press)
    keyboard_listener.start()

# Tray menu options
menu = Menu(
    MenuItem('Quit', on_quit)
)

# Create tray icon
icon = Icon("Activity Tracker", create_image(), menu=menu)

# AWS S3 integration setup
class ActivityTracker:
    def __init__(self, screenshot_interval=60, s3_bucket_name=None, s3_region='us-east-1'):
        self.screenshot_interval = screenshot_interval  # Interval for taking screenshots
        self.s3_bucket_name = s3_bucket_name  # S3 bucket name
        self.s3_region = s3_region  # Region of the S3 bucket
        
        # Initialize the S3 client
        self.s3_client = boto3.client('s3', region_name=self.s3_region)
        
        # Track the last screenshot time
        self.last_screenshot_time = time.time()

    def capture_screenshot(self):
        """Capture a screenshot, save it, and upload to S3."""
        current_time = time.time()
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        screenshot_filename = f"screenshot_{timestamp}.png"
        
        # Capture screenshot using pyautogui
        screenshot = pyautogui.screenshot()
        screenshot.save(screenshot_filename)
        
        log_activity("Screenshot Captured", f"Screenshot captured and saved as {screenshot_filename}")
        print(f"Screenshot captured: {screenshot_filename}")

        # Upload the screenshot to S3
        self.upload_file_to_s3(screenshot_filename)
        
        # Optionally delete the local file after uploading
        os.remove(screenshot_filename)
        
        # Reset last screenshot time
        self.last_screenshot_time = current_time

    def upload_file_to_s3(self, file_path):
        """Upload a file to S3."""
        if not self.s3_bucket_name:
            print("No S3 bucket name provided.")
            return
        
        try:
            # Upload the file to S3
            s3_object_key = f"screenshots/{os.path.basename(file_path)}"
            self.s3_client.upload_file(file_path, self.s3_bucket_name, s3_object_key)
            print(f"Uploaded {file_path} to S3 bucket {self.s3_bucket_name} under the key {s3_object_key}")
        except (NoCredentialsError, PartialCredentialsError) as e:
            print(f"Error with AWS credentials: {e}")
        except Exception as e:
            print(f"Error uploading file to S3: {e}")
    
    def capture_screenshots_periodically(self):
        """Periodically capture screenshots and upload them to S3."""
        while True:
            time.sleep(self.screenshot_interval)
            self.capture_screenshot()

# Main agent that integrates activity tracking and screenshot capturing
class BackgroundAgent:
    def __init__(self, s3_bucket_name, s3_region='us-east-1', screenshot_interval=60):
        self.s3_bucket_name = s3_bucket_name
        self.s3_region = s3_region
        self.screenshot_interval = screenshot_interval
        
        # Initialize the activity tracker for screenshots
        self.activity_tracker = ActivityTracker(screenshot_interval=screenshot_interval, s3_bucket_name=s3_bucket_name, s3_region=s3_region)
        
        # Start the activity listeners (mouse and keyboard)
        start_listeners()

        # Start the screenshot capturing thread
        self.screenshot_thread = threading.Thread(target=self.activity_tracker.capture_screenshots_periodically)
        self.screenshot_thread.start()

    def start(self):
        """Start the background agent."""
        icon.run()

# Example of how to use the BackgroundAgent
def main():
    # Set your S3 bucket name and region here
    s3_bucket_name = "your-s3-bucket-name"
    s3_region = "us-east-1"  # Change this to your region
    
    # Initialize and start the background agent
    agent = BackgroundAgent(s3_bucket_name=s3_bucket_name, s3_region=s3_region, screenshot_interval=60)
    agent.start()

if __name__ == "__main__":
    main()
